# Pre-processing of LAS tiles

Pre-processing consists of two parts:
1. Clipping of AHN point clouds to match target point clouds;
2. Ground and building surface extraction.

In [ ]:
%config Completer.use_jedi = False

# Add project src to path.
import set_path

# Import modules.
import numpy as np
import re
import pylas
import time

import src.preprocessing.ahn_preprocessing as ahn_preprocessing
import src.utils.las_utils as las_utils

## 1a. Clip a single file

In [ ]:
# The point cloud tile for which a corresponding AHN tile should be created.
filename = '../datasets/pointcloud/filtered_2386_9702.laz'
tile_code = re.match(r'.*(\d{4}_\d{4}).*', filename)[1]

In [ ]:
# Download the corresponding AHN cloud from 
# https://www.arcgis.com/apps/Embed/index.html?appid=a3dfa5a818174aa787392e461c80f781
!wget https://download.pdok.nl/rws/ahn3/v1_0/laz/C_25DN2.LAZ -P ../datasets/

In [ ]:
# Load the AHN point cloud.
# WARNING: this may require 16+ GB of RAM, depending on the size of the cloud.
ahn_cloud_file = '../datasets/C_25DN2.LAZ'
ahn_cloud = pylas.read(ahn_cloud_file)

In [ ]:
# Clip the corresponding AHN point cloud tile.
start = time.time()
ahn_preprocessing.clip_ahn_las_tile(ahn_cloud, filename, out_folder='../datasets/ahn/', buffer=1)
end = time.time()
print(f'Tile clipped in {end-start} seconds.')

## 1b. Clip all files in a folder

Note: we only provide one file, so this serves mainly as a howto rather than a real demonstration.

In [ ]:
las_folder = '../datasets/pointcloud/'
out_folder = '../datasets/ahn/'

# Set resume=True to skip existing files in the out_folder.
ahn_preprocessing.clip_ahn_las_folder(ahn_cloud, las_folder, out_folder=out_folder, buffer=1, resume=False)

## 2a. Surface extraction for a single file

In [ ]:
# Load the clipped tile that we have just created.
ahn_las_file = '../datasets/ahn/ahn_2386_9702.laz'

start = time.time()
# This will create the AHN ground and building surfaces, with a 0.1m resolution, and save them as an .npz file.
_file = ahn_preprocessing.process_ahn_las_tile(ahn_las_file, out_folder='../datasets/ahn/')
end = time.time()
print(f'Processed LAS tile in {end-start}s.')

# Load the result.
ahn_tile = ahn_preprocessing.load_ahn_tile(_file)

In [ ]:
# Optional: plot the resulting surfaces.

import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,6))
im1 = ax1.imshow(ahn_tile['ground_surface'].astype('float64'), interpolation='none')
im2 = ax2.imshow(ahn_tile['building_surface'].astype('float64'), interpolation='none')
ax1.set_title('Ground surface')
ax1.set_xticks(range(0, 501, 100))
ax1.set_xticklabels(np.arange(ahn_tile['x'][0], ahn_tile['x'][-1]+1, 10, dtype=int))
ax1.set_yticks(range(501, 0, -100))
ax1.set_yticklabels(np.arange(ahn_tile['y'][-1], ahn_tile['y'][0]+1, 10, dtype=int))
ax2.set_title('Building surface')
ax2.set_xticks(range(0, 501, 100))
ax2.set_xticklabels(np.arange(ahn_tile['x'][0], ahn_tile['x'][-1]+1, 10, dtype=int))
ax2.set_yticks([])
plt.colorbar(im1, ax=ax1, fraction=0.046, pad=0.04)
plt.colorbar(im2, ax=ax2, fraction=0.046, pad=0.04)
plt.show()

## 2b. Surface extraction for a folder of ahn cloud tiles

We have a batch script to do this, it allows using multiple CPU threads to speed things up.

    cd ../scripts
    python ahn_batch_processor.py --in_folder ../datasets/ahn/ --out_folder ../datasets/ahn/ --resume --workers 4